In [1]:
# Загрузка библиотек
import pandas as pd
import numpy as np

import scipy.stats as stats

In [2]:
# Загрузка данных
df_cleaned = pd.read_csv('data/df_cleaned.csv')
df_cleaned.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


## 1. Доверительный интервал

Посмотрим на процент конверсии по всей выборке.

In [3]:
df_cleaned.converted.mean()

0.11959585253297636

Какой процент отдельно для группы, которой показывалась новая посадочная страница, и группы, которой показывалась старая посадочная страница?

In [4]:
conv = df_cleaned.groupby('landing_page')['converted'].agg(['count', 'mean']).reset_index()
conv['mean'] = conv['mean'] * 100
conv

,landing_page,count,mean
0,new_page,145313,11.880561
1,old_page,145274,12.038630


Кажется, есть небольшое ухудшение в конверсии пользователей, которым показывалась новая посадочная страница. Для старой посадочной страницы конверсия - 12%, для новой 11,8%. Хочется узнать, значима ли эта разница?

Построим 95%-й доверительный интервал для разницы долей.

In [5]:
x = df_cleaned[df_cleaned.landing_page == 'new_page'].converted.values
y = df_cleaned[df_cleaned.landing_page == 'old_page'].converted.values

diff = x.mean() - y.mean() # разница средних
nx, ny = x.size, y.size # размеры долей
diff_std = np.sqrt(x.var(ddof=1)/nx + y.var(ddof=1)/nx) # стандартное отклонение долей

left, right = stats.norm.interval(0.95, loc=diff, scale=diff_std)
print(f'Доверительный интервал [{round(left, 5)}; {round(right, 5)}] ширины {round(right-left, 5)}')

Доверительный интервал [-0.00394; 0.00078] ширины 0.00472


Ноль входит в доверительный интервал, поэтому на уровне значимости 5% можно сделать вывод о том, что изменения не значимы.

Проверим гипотезу о равенстве средних, используя z-критерий.

Нулевая гипотеза H0: Средние не отличаются.

Альтернативная гипотеза H1: Существуют различия в средних значениях.

In [6]:
alpha = 0.05

z_obs = diff/diff_std # наблюдаемое значение z-критерия
z_crit = stats.norm.ppf(1 - alpha/2) # критическое значение z-критерия

print(f'{z_obs:.3} попадает в ({-z_crit:.3} , {z_crit:.3}) => Нулевая гипотеза НЕ отвергается')

-1.31 попадает в (-1.96 , 1.96) => Нулевая гипотеза НЕ отвергается


Наблюдаемое значение z-критерия попадает в критический интервал, значит на уровне значимости 5% можно сделать вывод о том, что изменения не значимы.

Зная наблюдаемое значение z-критерия, посчитаем p-уровень значимости.

In [7]:
print(f'p-уровень значимости = {2 * (1 - stats.norm().cdf(np.abs(z_obs)))}')

p-уровень значимости = 0.1891623336124968


p-уровень значимости больше 0.05, что также не позволяет отклонить гипотезу о равенстве средних.